## Import dataset

In [2]:
import pickle

with  open("diagrams_MNIST.pkl", "rb") as f:
    diagrams_MNIST = pickle.load(f)
    

In [5]:
labels = [0] * int(len(diagrams_MNIST)/2) + [1] * int(len(diagrams_MNIST)/2)

In [7]:
diags_dict ={'diagrams' : diagrams_MNIST}


In [10]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import *
from sklearn.svm import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, KFold, ShuffleSplit
from tensorflow import random_uniform_initializer as rui
import gudhi.representations as tda

### Uncomment the following to process your diagrams (necessary)
thresh = 500

# Whole pipeline
tmp = Pipeline([
        ("Selector",      tda.DiagramSelector(use=True, point_type="finite")),
        ("ProminentPts",  tda.ProminentPoints(use=True, num_pts=thresh)),
        ("Scaler",        tda.DiagramScaler(use=True, scalers=[([0,1], MinMaxScaler())])),
        ("Padding",       tda.Padding(use=True)),
                ])

prm = {filt: {"ProminentPts__num_pts": min(thresh, max([len(dgm) for dgm in diags_dict[filt]]))} 
       for filt in diags_dict.keys() if max([len(dgm) for dgm in diags_dict[filt]]) > 0}

# Apply the previous pipeline on the different filtrations.
diags = []
for dt in prm.keys():
    param = prm[dt]
    tmp.set_params(**param)
    diags.append(tmp.fit_transform(diags_dict[dt]))

# For each filtration, concatenate all diagrams in a single array.
D, npts = [], len(diags[0])
for dt in range(len(prm.keys())):
    D.append(np.array(np.concatenate([diags[dt][i][np.newaxis,:] for i in range(npts)],axis=0),dtype=np.float32))
